In [1]:
import dash
from dash import dcc,html,Dash
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import pandas as pd

In [2]:
from sqlalchemy.sql import exists  
from main import Session,Base,engine,Menu,Hotels
from sqlalchemy import and_
local_session=Session(bind=engine)
con=engine.connect()

In [3]:
#converts sql tables to pandas dataframe: pandas.core.frame.DataFrame

foods= pd.read_sql_table("Menu", engine)
hotels= pd.read_sql_table("Hotels", engine)

2022-02-26 13:16:28,039 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-26 13:16:28,047 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 13:16:28,055 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2022-02-26 13:16:28,055 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 13:16:28,063 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Menu")
2022-02-26 13:16:28,071 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 13:16:28,079 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-26 13:16:28,079 INFO sqlalchemy.engine.Engine [raw sql] ('Menu',)
2022-02-26 13:16:28,087 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Menu")
2022-02-26 13:16:28,087 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 13:16:28,095 INFO sqlalchemy.engine.Engine SELECT sql F

In [4]:
foods["foodname"] = foods["foodname"].str.lower()
foods.head()

,index,hotel_id,hotelname,food_id,foodname,foodType,price
0,1,2,Im Thirsty,123.0,chilli garlic al- faham,Non-Veg,169
1,2,2,Im Thirsty,124.0,spicy pepper al- faham,Non-Veg,169
2,3,2,Im Thirsty,125.0,coal barbecue,Non-Veg,179
3,4,2,Im Thirsty,126.0,special grill,Non-Veg,159
4,5,2,Im Thirsty,127.0,hariyali tikka,Non-Veg,79


In [5]:
hotels.head()

,hotel_id,hotelname,speciality,pureVeg,city,location,link,rating,costfor2,scrapped
0,0,Alif Paradise Biriyani,"Biryani, Indian",None,peelamedu,"Peelamedu, Stv Nagar",https://www.swiggy.com/restaurants/alif-paradi...,3.5,200.0,True
1,2,Im Thirsty,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",https://www.swiggy.com/restaurants/im-thirsty-...,NaN,200.0,True
2,3,CREAMERY,Italian,None,peelamedu,"Peelamedu, Peelamedu",https://www.swiggy.com/restaurants/creamery-pe...,4.5,500.0,True
3,4,Keventers - Milkshakes and Desserts,"Beverages, Ice Cream",Pure Veg,peelamedu,"peelamedu, Peelamedu",https://www.swiggy.com/restaurants/keventers-m...,4.4,300.0,True
4,5,Mr Grill,"Grill, Indian",None,peelamedu,"Peelamedu, Peelamedu",https://www.swiggy.com/restaurants/mr-grill-pe...,4.0,250.0,True


In [6]:
hotels["rating"]=hotels["rating"].replace(np.NaN, 0)
hotels=hotels.drop("hotelname",axis=1)
df=pd.merge(foods,hotels,on="hotel_id")
df=df.drop(["scrapped","link","food_id"],axis=1)
df

,index,hotel_id,hotelname,foodname,foodType,price,speciality,pureVeg,city,location,rating,costfor2
0,1,2,Im Thirsty,chilli garlic al- faham,Non-Veg,169,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",0.0,200.0
1,2,2,Im Thirsty,spicy pepper al- faham,Non-Veg,169,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",0.0,200.0
2,3,2,Im Thirsty,coal barbecue,Non-Veg,179,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",0.0,200.0
3,4,2,Im Thirsty,special grill,Non-Veg,159,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",0.0,200.0
4,5,2,Im Thirsty,hariyali tikka,Non-Veg,79,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",0.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...
110236,108448,81,The Table,almost baked jar,Non-Veg,209,Restaurant ClosedDesserts,None,peelamedu,"Inside Zucca pizza, Peelamedu",0.0,400.0
110237,108449,81,The Table,raspberry truffle shuffle jar,Veg,209,Restaurant ClosedDesserts,None,peelamedu,"Inside Zucca pizza, Peelamedu",0.0,400.0
110238,108450,81,The Table,black forest jar,Veg,209,Restaurant ClosedDesserts,None,peelamedu,"Inside Zucca pizza, Peelamedu",0.0,400.0
110239,108451,81,The Table,black velvet jar,Non-Veg,209,Restaurant ClosedDesserts,None,peelamedu,"Inside Zucca pizza, Peelamedu",0.0,400.0


In [17]:
#app displays food searched around that city along with rating price hotel name, food name

app1=dash.Dash()

app1.layout=html.Div(children=
                    [
                        html.H1('Swiggy Data Analysis', style={'text-align':'center'}),
                        html.Div(
                            [html.H4('Select Area : ',style={'display':'inline-block','text-align':'center'}),
                            dcc.Dropdown(id="slct_area",
                             options=[
                                     {"label": "Ganapathy", "value": "Ganapathy"},
                                     {"label": "gandhipuram", "value": "gandhipuram"},
                                     {"label": "hopes", "value": "hopes"},
                                     {"label": "peelamedu", "value": "peelamedu"},
                                     {"label": "race-course", "value": "race-course"},
                                     {"label": "ramanathapuram", "value": "ramanathapuram"},
                                     {"label": "rs-puram", "value": "rs-puram"},
                                     {"label": "sai-baba-colony", "value": "sai-baba-colony"},
                                     {"label": "saravanampatti", "value": "saravanampatti"}
                                 ],
                                 multi=False,
                                 value="saravanampatti",
                                 style={'width': "40%",'margin-top':'-50px',
                                        'margin-left':'65px',"height":'40px','align':'center'}
                                 )
                            ]),
                        
                        html.Div([html.H4('Search : ',style={'display':'inline-block','align':'center'}),
                          dcc.Input(id="input",value="idly",
                        style={'display':'inline-block','width': '30%',"height":'30px','margin-top':'20px','margin-left':'10px'})
                         ]),
                        
                        
                        html.H1(' '),
                        html.Div(id="output")
                    ])


@app1.callback(
     Output(component_id='output',component_property="children"),
    [Input(component_id='slct_area', component_property="value"),
     Input(component_id="input",component_property="value")])

def update_value(input_data2,input_data1):
    input_data1=input_data1.lower()
    strr=input_data1.split(" ")
    df1=df[df.foodname.apply(lambda s: all(words in s for words in strr))]
    df1=df1[df1["city"]==input_data2]
    return dcc.Graph(id="foods",
            figure=px.bar(
            data_frame=df1,
            x='hotelname',
            y='price',
            hover_data=['foodname', 'price','rating'],
            template='ggplot2'
    ))


In [18]:
if __name__=="__main__":
    app1.run_server(port=3001)

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3001/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2022 13:27:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:19] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:20] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 13:27:20] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


In [24]:
hotels1=pd.read_sql_table("Hotels", engine)
hotels1

2022-02-26 16:53:49,567 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-26 16:53:49,579 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 16:53:49,600 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' ORDER BY name
2022-02-26 16:53:49,602 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 16:53:49,607 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Hotels")
2022-02-26 16:53:49,609 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 16:53:49,618 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-26 16:53:49,620 INFO sqlalchemy.engine.Engine [raw sql] ('Hotels',)
2022-02-26 16:53:49,626 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Hotels")
2022-02-26 16:53:49,628 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-26 16:53:49,631 INFO sqlalchemy.engine.Engine PRAGMA

,hotel_id,hotelname,speciality,pureVeg,city,location,link,rating,costfor2,scrapped
0,0,Alif Paradise Biriyani,"Biryani, Indian",None,peelamedu,"Peelamedu, Stv Nagar",https://www.swiggy.com/restaurants/alif-paradi...,3.5,200.0,True
1,2,Im Thirsty,"Fast Food, Beverages",None,peelamedu,"peelamedu, peelamedu",https://www.swiggy.com/restaurants/im-thirsty-...,NaN,200.0,True
2,3,CREAMERY,Italian,None,peelamedu,"Peelamedu, Peelamedu",https://www.swiggy.com/restaurants/creamery-pe...,4.5,500.0,True
3,4,Keventers - Milkshakes and Desserts,"Beverages, Ice Cream",Pure Veg,peelamedu,"peelamedu, Peelamedu",https://www.swiggy.com/restaurants/keventers-m...,4.4,300.0,True
4,5,Mr Grill,"Grill, Indian",None,peelamedu,"Peelamedu, Peelamedu",https://www.swiggy.com/restaurants/mr-grill-pe...,4.0,250.0,True
...,...,...,...,...,...,...,...,...,...,...
1287,1288,BUBBLY APPE,"Fast Food, Snacks",None,saravanampatti,"Saravanampatti, Saravanampatti",https://www.swiggy.com/restaurants/bubbly-appe...,NaN,150.0,True
1288,1289,Wrap & Eat,"Arabian, Barbecue",None,ramanathapuram,"Ramanathapuram, Ramanathapuram",https://www.swiggy.com/restaurants/wrap-and-ea...,NaN,200.0,True
1289,1290,BAKES & BYTES,Bakery,None,ramanathapuram,"Ramanathapuram, Ramanathapuram",https://www.swiggy.com/restaurants/bakes-and-b...,NaN,150.0,True
1290,1291,Chickburg,cks,None,Ganapathy,"Coi Ganapathy, Coi Ganapathy",https://www.swiggy.com/restaurants/chickburg-c...,NaN,200.0,True


In [33]:
app2=dash.Dash()

app2.layout=html.Div(children=
                    [
                        html.H1('Swiggy Data Analysis', style={'text-align':'center'}),
                        html.Div(
                            [html.H4('Select Area : ',style={'display':'inline-block','text-align':'center'}),
                            dcc.Dropdown(id="slct_area",
                             options=[
                                     {"label": "Ganapathy", "value": "Ganapathy"},
                                     {"label": "gandhipuram", "value": "gandhipuram"},
                                     {"label": "hopes", "value": "hopes"},
                                     {"label": "peelamedu", "value": "peelamedu"},
                                     {"label": "race-course", "value": "race-course"},
                                     {"label": "ramanathapuram", "value": "ramanathapuram"},
                                     {"label": "rs-puram", "value": "rs-puram"},
                                     {"label": "sai-baba-colony", "value": "sai-baba-colony"},
                                     {"label": "saravanampatti", "value": "saravanampatti"}
                                 ],
                                 multi=False,
                                 value="saravanampatti",
                                 style={'width': "40%",'margin-top':'-50px',
                                        'margin-left':'65px',"height":'40px','align':'center'}
                                 )
                            ]),
                        html.H1(' '),
                        html.Div(id="output1"),
                        
                        html.Div([html.H4('Search : ',style={'display':'inline-block','align':'center'}),
                          dcc.Input(id="input",value="idly",
                        style={'display':'inline-block','width': '30%',"height":'30px','margin-top':'20px','margin-left':'10px'})
                         ]),
                        
                        
                        html.H1(' '),
                        html.Div(id="output")
                    ])


@app2.callback(
     [   Output(component_id='output1',component_property="children"),
         Output(component_id='output',component_property="children")],
    [Input(component_id='slct_area', component_property="value"),
     Input(component_id="input",component_property="value")])

def update_value(input_data2,input_data1):
    input_data1=input_data1.lower()
    strr=input_data1.split(" ")
    df1=df[df.foodname.apply(lambda s: all(words in s for words in strr))]
    df1=df1[df1["city"]==input_data2]
    df2=hotels1
    g2=dcc.Graph(id="foods",
                figure=px.bar(
                data_frame=df1,
                x='hotelname',
                y='price',
                hover_data=['foodname', 'price','rating'],
                template='ggplot2'))
    g1= dcc.Graph(id="hotels",
        figure=px.scatter(df2, x="costfor2",y="rating", hover_data=["hotelname","speciality"],template='ggplot2',color="speciality"))
    
    #fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
    #                 size="pop", color="continent", hover_name="country",
    #                 log_x=True, size_max=55)

    #fig.update_layout(transition_duration=500)
    return (g1,g2)
    
   


            


In [34]:
if __name__=="__main__":
    app2.run_server(port=3001)

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

Dash is running on http://127.0.0.1:3001/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3001/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2022 17:01:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:58] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 17:01:58] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


In [10]:
app=dash.Dash()
app.layout=html.Div(children=
[
    html.H1('Price Comparison Graph'),
    html.Div([html.H4('Filter by Rating : ',style={'display':'inline-block'}),
              dcc.Dropdown(id="slct_rating",
                 options=[
                     {"label":"None", "value": 0},
                     {"label":1, "value": 1},
                     {"label":2, "value": 2},
                     {"label":3, "value": 3},
                     {"label":4, "value": 4},
                 ],
                 value=0,
                 multi=False,
                 style={'width': '40%','margin-top':'-50px','margin-left':'65px',"height":'40px'}
                 )
             ]
            ),

    html.Div([html.H4('Search : ',style={'display':'inline-block'}),
              dcc.Input(id="input",value="Search foods",type="text",
                        style={'display':'inline-block','width': '30%',"height":'30px','margin-top':'20px','margin-left':'10px'})
             ]),
    html.H1(' '),
    html.Div(id="output")
    
])

@app.callback(
    
        Output(component_id="output",component_property="children"),
    [Input(component_id='slct_rating', component_property='value'),
     Input(component_id="input",component_property="value")])

def update_value(input_data2,input_data1):
    input_data1=input_data1.lower()
    strr=input_data1.split(" ")
    df1=df[df.foodname.apply(lambda s: all(words in s for words in strr))]
    df1=df1[df1["rating"]>=input_data2]
    return dcc.Graph(id="foods",
            figure=px.bar(
            data_frame=df1,
            x='hotelname',
            y='price',
            hover_data=['foodname', 'price','rating'],
            template='plotly_dark'
    ))

In [11]:
if __name__=="__main__":
    app.run_server(port=3000)

Dash is running on http://127.0.0.1:3000/

Dash is running on http://127.0.0.1:3000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2022 11:04:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_2_0m1645588313.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_2_0m1645588313.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/deps/react@16.v2_2_0m1645588313.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_2_0m1645588313.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_2_0m1645588313.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_2_0m1645588313.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 11:04:24] "GET /_dash-component-suites/d

In [4]:
hot1=hotels.groupby(["city"]).count()
print(hot1[2:])

                 hotel_id  hotelname  speciality  pureVeg  location  link  \
city                                                                        
hopes                 178        178         178       21       178   178   
peelamedu             146        146         146       19       146   146   
race-course            70         70          70       16        70    70   
ramanathapuram        125        125         125       18       125   125   
rs-puram              230        230         230       61       230   230   
sai-baba-colony       161        161         161       41       161   161   
saravanampatti        148        148         148       16       148   148   

                 rating  costfor2  scrapped  
city                                         
hopes               116       178       178  
peelamedu            93       146       146  
race-course          48        70        70  
ramanathapuram       80       125       125  
rs-puram            123       

In [5]:
hot1



,hotel_id,hotelname,speciality,pureVeg,location,link,rating,costfor2,scrapped
city,,,,,,,,,
Ganapathy,107,107,107,9,107,107,55,107,107
gandhipuram,127,127,127,24,127,127,71,127,127
hopes,178,178,178,21,178,178,116,178,178
peelamedu,146,146,146,19,146,146,93,146,146
race-course,70,70,70,16,70,70,48,70,70
ramanathapuram,125,125,125,18,125,125,80,125,125
rs-puram,230,230,230,61,230,230,123,230,230
sai-baba-colony,161,161,161,41,161,161,88,160,161
saravanampatti,148,148,148,16,148,148,95,148,148


In [6]:
app = dash.Dash()

In [10]:
app.layout=html.Div(children=[
    html.H1("Restauants around coimbatore...", style={'text-align':'center'}),
    
    dcc.Dropdown(id="slct_area",
                 options=[
                     {"label": "Ganapathy", "value": "Ganapathy"},
                     {"label": "gandhipuram", "value": "gandhipuram"},
                     {"label": "hopes", "value": "hopes"},
                     {"label": "peelamedu", "value": "peelamedu"},
                     {"label": "race-course", "value": "race-course"},
                     {"label": "ramanathapuram", "value": "ramanathapuram"},
                     {"label": "rs-puram", "value": "rs-puram"},
                     {"label": "sai-baba-colony", "value": "sai-baba-colony"},
                     {"label": "saravanampatti", "value": "saravanampatti"}
                 ],
                 multi=False,
                 value="saravanampatti",
                 style={'width': "40%",'margin-top':'-50px',
                        'margin-left':'65px',"height":'40px'}
                 ),
    
    html.Br(),
    
    dcc.Graph(id='my_bee_map', figure={})

])




In [8]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_area', component_property='value')]
)

SyntaxError: unexpected EOF while parsing (<ipython-input-8-a9d69c9c8ed9>, line 5)

In [12]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

In [11]:
from dash import Dash, html, dcc

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal')
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
